In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
import json

In [2]:
def get_semo_monthly_caps():
    # function get's the latest registered capacity
    
    base = 'https://reports.sem-o.com/documents/'
    suffix1 = 'PUB_MnlyRegisteredCapacity_{0}{1}010900.xml'
    suffix2 = 'PUB_MnlyRegisteredCapacity_{0}{1}010900.xml'
    
    now = dt.datetime.utcnow()
    
    # sometimes report is published at 10 am instead of 9am - account for this
    try:
        df_cap = pd.read_xml(base + suffix1.format(str(now.year),str(now.month).zfill(2)))
    except:
        df_cap = pd.read_xml(base + suffix2.format(str(now.year),str(now.month).zfill(2)))
        
    return df_cap.set_index('ResourceName')


def get_isem_solar_units():
    # function returns dataframe with registed solar unit data
    df_cap = get_semo_monthly_caps()
    
    semo_url = 'https://www.sem-o.com/documents/general-publications/'
    file = 'List-of-Registered-Units.xlsx'
    
    df_units = pd.read_excel(semo_url + file,
                             sheet_name = 'Registered Units TSC',
                             skiprows=2
                            ).set_index('Resource Name')

    df_units = df_units.loc[df_units['Fuel Type'] == 'SOLAR']

    df_units = df_units.merge(df_cap[['RegisteredCapacity','Jurisdiction']],
                              left_index=True,
                              right_index=True)
    
    return df_units.drop_duplicates()


def build_req_url(start,end,unit):
    # function builds the url to query unit metered generation via SEMO api
    base = 'https://reports.sem-o.com/api/v1/dynamic/BM-086?'
    times = 'StartTime=%3E%3D{0}T00%3A00%3A00%3C%3D{1}T23%3A59%3A00'
    other = '&sort_by=StartTime&order_by=ASC'
    units = '&ParticipantName=&ResourceName={0}'
    pagination = '&page=1&page_size=5000'
    
    return base + times.format(start,end) + other + units.format(unit) + pagination


def get_unit_mg(start, end, unit):
    # get the metered generation for a given unit
    dates = pd.date_range(start=start,
                      end= end,
                      freq = 'D').strftime('%Y-%m-%d')
    
    
    dfs = []
    
    for i in range(0,len(dates)-1):
        
        start = dates[i]
        end = dates[i+1]
        
        url = build_req_url(start,end,unit)
        
        r = requests.get(url)
        df = pd.DataFrame(json.loads(r.text)['items']).drop_duplicates()
        dfs.append(df)
        
    return pd.concat(dfs).drop_duplicates()


def get_all_solar_units(start,end):
    # get the metered gen for all solar units
    df_units = get_isem_solar_units()
    
    dfs = []
    for unit in df_units.index:
        dfs.append(get_unit_mg(start, end, unit))
    
    df_solar = pd.concat(dfs)
    df_solar['StartTime'] = pd.to_datetime(df_solar['StartTime'])
    
    df_solar = 2*df_solar.groupby(['StartTime','ResourceName'])['MeteredMW'].mean()
    
    return df_solar.unstack()
      


In [ ]:
start = dt.datetime(year = 2023,
                   month = 1,
                   day = 1)

end = dt.datetime.utcnow()


df_solar = get_all_solar_units(start,end)


df_solar.head()